In [2]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats 

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
df_catinfo = pd.read_csv('../data/DSA3101_Hackathon_Categories_Information.csv')
df_hackdata = pd.read_csv('../data/DSA3101_Hackathon_Data.csv', parse_dates=['Date'])
df_paneldemo = pd.read_excel('../data/DSA3101_Hackathon_Panelists_Demographics.xlsx')

FileNotFoundError: ignored

In [ ]:
print(df_catinfo.info())
df_catinfo.head()

In [ ]:
df_catinfo.Category.unique().shape[0]

In [ ]:
print(df_hackdata.info())
df_hackdata.head()

In [ ]:
df_hackdata.Date.max()

In [ ]:
df_hackdata.Category.unique().shape[0]

In [ ]:
fig, ax = plt.subplots(2, 3, figsize = (30, 20), sharey= False)
sns.countplot(df_hackdata['Pack Size'], ax = ax[0, 0])
sns.countplot(df_hackdata['Pack Size'], ax = ax[1, 0])
ax[1, 0].set(xlim = (0, 6))
sns.countplot(df_hackdata['Volume'], ax=ax[0, 1])
sns.countplot(df_hackdata['Volume'], ax=ax[1, 1])
ax[1, 1].set(xlim = (0, 10))
sns.countplot(df_hackdata['Spend'], ax=ax[0, 2])
sns.countplot(df_hackdata['Spend'], ax=ax[1, 2])
ax[1, 2].set(xlim = (0, 10))
plt.show()

* format Date

In [ ]:
# Zero Pack Size
df_hackdata[df_hackdata['Pack Size'] == 0]

In [ ]:
# Zero Volume
df_hackdata[df_hackdata.Volume == 0]


In [ ]:
## Zero Spenders
df_hackdata[df_hackdata.Spend == 0]

In [ ]:
print(df_paneldemo.info())
df_paneldemo.head()

In [ ]:
df_paneldemo.apply(lambda x: x.unique().shape[0], axis = 0)

In [ ]:
df_paneldemo['Ethnic'] = df_paneldemo.Ethnicity.apply(lambda x: x.split(" ")[-1])
df_paneldemo

In [ ]:
df_paneldemo.to_csv('../data/DSA3101_Hackathon_Panelists_Demographics.csv')

## Unique Values 

In [ ]:
#df_paneldemo.apply(lambda x: x.unique()[0:2].append([pd.NaN]), axis = 0)

df_paneldemo.apply(lambda x: pd.Series(x.unique()[0:12]), axis = 0).fillna('')

## RFM

Each visit of the same client on the same day is splitted into different entries, this might lead to wrong calculation that a client has visited the place many times

In [ ]:
df_hackdata.sort_values(['Panel ID', 'Date'], ascending = [True, False]).head(10)

In [ ]:
def recency0(group):
    # get the most recent 3 purchase and sum their time away, 
    # penalise customers who hasnt buy for very long time
    
    # redefine an 'effective visit' as a visit whereby a customer
    # spend more than half of what he or she usually spend
    recent3 = date_series.sort_values(ascending = False)[:3]
    day = (pd.Timestamp.today() - recent3).apply(lambda x: x.days).tolist()
    
    score = 0
    for i, e in enumerate(day):
        score += e**(i+1)
    
    return score

def recency1(group):
    # redefine an 'effective visit' as a visit whereby a customer
    # spend more than half of what he or she usually spend
    spending = group['Spend']
    date = group['Date']
    datespend = tuple(zip(date, spending))
    datespend = sorted(datespend, key = lambda x: x[0], reverse = True)
    
    for date, spend in datespend:
        if stats.percentileofscore(spending, spend) >= 90:
            effective_day = (pd.Timestamp.today() - date).days
            return effective_day

def frequency(date_series):
  return len(date_series)

def monetary(spend_series):
  return sum(spend_series)

In [ ]:
# collaps each visit into one row
df_hackdata_grouped = df_hackdata.groupby(['Panel ID', 'Date']).agg({"Spend": sum})
df_hackdata_grouped = df_hackdata_grouped.reset_index()
b = df_hackdata_grouped
b = b.groupby('Panel ID')

In [ ]:
a = b.agg({'Date': frequency,
           'Spend': monetary})
a = a.rename(columns = {'Date': 'frequency', 'Spend': 'monetary'})
a['recency'] = b.apply(recency1)
a = a.fillna(1000)
a.head()

In [ ]:
a['R'] =  pd.qcut(a.recency, q = 5, labels = range(5, 0, -1)).astype('int32')
a['F'] =  pd.qcut(a.frequency, q = 5, labels = range(1, 6, 1)).astype('int32')
a['M'] =  pd.qcut(a.monetary, q = 5, labels = range(1, 6, 1)).astype('int32')
a['RFM'] =  a.apply(lambda x: x.R*100 + x.F*10 + x.M, axis = 1)
a.head()

In [ ]:
a.to_csv('../data/panel_RFM.csv')

## leaving customers

Customers who are to leave are likely those who are high in frequency, indicating that they were once active, but low in recency, indicating that they havent purchase from the shop for awhile. Then rank them in their M value to rate their priority to the business.  

From the tableau visualisation and the sorting we can see that the recency score is highly right-skewed(most of the customers are in the vewry recent category).   
*We might need more efficient ways to rating R criteria)*   

Since most of the customers are in comes with R = 5, it is a strong signal that a customer is leaving if their R is lower than 5.

In [ ]:
a.R.value_counts()

In [ ]:
a.F.value_counts()

In [ ]:
a.M.value_counts()

In [ ]:

colors = ['orange',] + ['grey']* 4
explode = (0.1, 0, 0, 0, 0)  # explode 1st slice

# Plot
fig1, ax1 = plt.subplots(figsize = (8,8))
ax1.pie(a.R.value_counts(), 
        labels = a.R.value_counts().index.tolist(),
        explode=explode, autopct='%1.1f%%', shadow=True, startangle=140,
        textprops={'fontsize': 14},
       colors = colors)

plt.legend(loc="best", fontsize = 15)
plt.axis('equal')
plt.show()

In [ ]:
print(a.recency.describe())
sns.distplot(a.recency)

In [ ]:
non_R5 = a[a.R != 5]
non_R5.sort_values(["F", 'R', 'M'], ascending = [False, False, False]).head(20)

## Customer Segmentation

In [ ]:
rfm_distribution = a.groupby(['RFM']).size().reset_index(name='counts').\
sort_values(ascending=False, by='counts', ignore_index=True)
rfm_distribution['percentage'] = rfm_distribution['counts']/rfm_distribution['counts'].sum()*100
rfm_distribution= rfm_distribution.drop(columns=['counts'])

rfm_distribution